# Data preparation

In [2]:
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import os
import sys

import tiktoken
import openai

from langdetect import detect, detect_langs, DetectorFactory

from tqdm.asyncio import tqdm
from tqdm.asyncio import tqdm as atqdm

import logging
import asyncio
import aiofiles
import time
import json
import re

from rapidfuzz import fuzz


In [3]:
load_dotenv()
client = openai.OpenAI()
client_async = openai.AsyncOpenAI()

csv = pd.read_csv('./jobs_data_clean.csv')

sample_sections_test = csv.copy()
sample_sections_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer


## Determine the language of vacancies

### `langdetect` library

In [4]:
DetectorFactory.seed = 0  # We record the result so that there are no accidental changes

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"
    
def detect_language_with_confidence(text):
    try:
        lang_probs = detect_langs(text)
        if lang_probs:
            return str(lang_probs[0])  # Format: 'en:0.99'
    except:
        return "unknown"

#df["Language langdetect"] = df["Job Description"].apply(detect_language)
#df["Language langdetect confidence"] = df["Job Description"].apply(detect_language_with_confidence)

In [5]:
#df.to_csv('./jobs_data_langdetect.csv', index=False)
csv_1 = pd.read_csv('./jobs_data_langdetect.csv')
sample_sections_test = csv_1.copy()

In [6]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Derive column 'derivedCol' from column: 'Language langdetect confidence'
    # Transform based on the following examples:
    #    Language langdetect confidence    Output
    # 1: "en:0.9999973661975282"        => "0.9999973661975282"
    df.insert(20, "derivedCol", df["Language langdetect confidence"].str.split(":").str[-1])
    # Change column type to float64 for column: 'derivedCol'
    df = df.astype({'derivedCol': 'float64'})
    # Drop column: 'Language langdetect confidence'
    df = df.drop(columns=['Language langdetect confidence'])
    # Rename column 'derivedCol' to 'Language langdetect confidence'
    df = df.rename(columns={'derivedCol': 'Language langdetect confidence'})
    return df

sample_sections_test = clean_data(sample_sections_test.copy())
sample_sections_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language langdetect,Language langdetect confidence
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,0.999997
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999996
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999994
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999997
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en,0.999996


### Confidence lower than 0.99

In [7]:
"""
Cell generated by Data Wrangler.
"""
def low_confidence(df):
    # Sort by column: 'Language langdetect confidence' (ascending)
    df = df.sort_values(['Language langdetect confidence'])
    # Filter rows based on column: 'Language langdetect confidence'
    df = df[df['Language langdetect confidence'] < 0.99]
    return df

not_enough_confidence = low_confidence(sample_sections_test.copy())
sample_sections_test = sample_sections_test.drop(not_enough_confidence.index)


In [8]:
"""
Cell generated by Data Wrangler.
"""
def clean_data1(df):
    # Drop column: 'Language langdetect confidence'
    df = df.drop(columns=['Language langdetect confidence'])
    # Rename column 'Language langdetect' to 'Language'
    df = df.rename(columns={'Language langdetect': 'Language'})
    return df

sample_sections_test = clean_data1(sample_sections_test.copy())
sample_sections_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en
1,Austria,Europe,True,True,default,google.com,Senior Ios Developer,Pyramid Global Technologies,Austria,"Trabajo.org - Stellenangebote, Arbeit, StudySm...",A minimum of 6+ years of concurrent commercial...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTZW5pb3IgSW9zIERldmVsb3Blci...,2025-01-13 12:20:43 UTC,iOS developer,en
2,Austria,Europe,True,True,default,google.com,iOS Developer - Permanent remote,Bluestorm Recruitment by Dazzle,Austria,Jooble,iOS Developer\r\n\r\nOur client is a leading m...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIC0gUGVybW...,2025-01-13 12:20:43 UTC,iOS developer,en
3,Austria,Europe,True,True,default,google.com,Sr. Ios Developer,Bykon,Austria,"Trabajo.org - Stellenangebote, Arbeit",In ByKon we're looking for an exceptional Sr.\...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTci4gSW9zIERldmVsb3BlciIsIm...,2025-01-13 12:20:43 UTC,iOS developer,en
4,Austria,Europe,True,True,default,google.com,Software Engineer/ iOS,Bitpanda,Anywhere,GrabJobs,Who we are\r\n\r\nWe simplify wealth creation....,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJTb2Z0d2FyZSBFbmdpbmVlci8gaU...,2025-01-13 12:20:43 UTC,iOS developer,en


### Chat GPT API

In [9]:
# Configure logging
logging.basicConfig(level=logging.WARNING, format="%(asctime)s - %(levelname)s - %(message)s")

# Suppress OpenAI API HTTP logs
logging.getLogger("httpx").setLevel(logging.WARNING)



async def check_rate_limits(response_headers, rpm):
    """Checks OpenAI API rate limits and waits if needed, without processing reset time."""

    # Extract limits from headers
    limit_requests = int(response_headers.get("x-ratelimit-limit-requests", rpm))  # Default to given RPM if missing
    limit_tokens = int(response_headers.get("x-ratelimit-limit-tokens", 0))  # TPM is always provided

    remaining_requests = int(response_headers.get("x-ratelimit-remaining-requests", limit_requests))
    remaining_tokens = int(response_headers.get("x-ratelimit-remaining-tokens", limit_tokens))

    # If RPD or TPD is reached → Stop execution
    if remaining_requests == 0:
        print(f"🔴 Request limit (RPD) reached! Execution stopped.")
        return False  # Stop execution

    if remaining_tokens == 0:
        print(f"🔴 Token limit (TPD) reached! Execution stopped.")
        return False  # Stop execution

    # If RPM or TPM is reached → Wait 60 seconds and continue
    if remaining_requests < 1 or remaining_tokens < 1:
        print(f"🟡 API rate limit reached! Waiting 60 seconds...")
        await asyncio.sleep(60)

    return True  # Continue execution



async def chatgpt_async(
    input_column_name=None,
    output_column_name=None,
    input_text_length=None,
    output_text_length=5,
    num_rows=None,
    df=None,
    system_prompt=None,
    user_prompt=None,
    gpt_model=None,
    client=None,
    batch_size=None,
    cache_file=None,
    #rpm=500,
    concurrency_limit=10,
    max_retries=5,
    retry_delay=0.125
):
    """Async function to process API requests in batches and cache responses."""

    # Input Validations
    if df is None:
        raise ValueError("df (dataframe) must be provided.")
    if input_column_name is None:
        raise ValueError("input_column_name must be provided.")
    if input_column_name not in df.columns:
        raise ValueError(f"{input_column_name} column does not exist in the dataframe.")
    if output_column_name is None:
        raise ValueError("output_column_name must be provided.")
    if gpt_model is None:
        raise ValueError("gpt_model must be provided.")
    if user_prompt is None:
        raise ValueError("user_prompt must be provided.")
    if client is None:
        raise ValueError("client must be provided.")

    # Prepare Data
    df = df.head(num_rows).copy() if num_rows else df.copy()

    if output_column_name not in df.columns:
        df[output_column_name] = ""

    # Load Cache if Enabled
    cache = {}
    if cache_file and os.path.exists(cache_file):
        async with aiofiles.open(cache_file, "r", encoding="utf-8") as f:
            try:
                cache = json.loads(await f.read())
            except json.JSONDecodeError:
                logging.warning("Cache file is corrupted or empty. Proceeding without cache.")

    semaphore = asyncio.Semaphore(concurrency_limit)  # Limit concurrent requests

    async def process_row(index, row):
        """Asynchronously process each row with caching and retry support."""
        async with semaphore:
            column_text = row[input_column_name]
            truncated_text = " ".join(column_text.split()[:input_text_length]) if input_text_length else column_text

            # Check if result is cached
            if cache_file and truncated_text in cache:
                df.at[index, output_column_name] = cache[truncated_text]
                return  # Skip API call
            
            # New version
            prompt = f"{user_prompt} {truncated_text}"

            messages = []
            if system_prompt:
                messages.append({"role": "system", "content": system_prompt})
            messages.append({"role": "user", "content": prompt})

            for attempt in range(max_retries):
                try:
                    # Fetch response with raw headers
                    response = await client.chat.completions.with_raw_response.create(
                        model=gpt_model,
                        messages=messages,
                        max_tokens=output_text_length,
                        temperature=0
                    )

                    # Check API limits using response headers
                    #if not await check_rate_limits(response.headers, rpm):
                    #    return  # Stop processing if RPD/TPD is reached

                    response_text = response.parse().choices[0].message.content

                    if cache_file:
                        cache[truncated_text] = response_text
                    df.at[index, output_column_name] = response_text
                    return  

                except Exception as e:
                    error_message = str(e)

                    # Try extracting wait time in milliseconds or seconds
                    wait_time_match = re.search(r'Please try again in (\d+(?:\.\d+)?)\s*(ms|s)', error_message)

                    if wait_time_match:
                        wait_time_value = float(wait_time_match.group(1))  # Extract the numeric value
                        wait_time_unit = wait_time_match.group(2)  # Extract unit (ms or s)

                        # Convert milliseconds to seconds if needed
                        dynamic_retry_delay = wait_time_value / 1000 if wait_time_unit == "ms" else wait_time_value
                    else:
                        dynamic_retry_delay = retry_delay  # Default fallback

                    # Add a small buffer (e.g., 5ms extra wait time)
                    dynamic_retry_delay += 0.005

                    if attempt < max_retries - 1:
                        #logging.warning(f"Error on row {index}, attempt {attempt+1}: {error_message}. Retrying in {dynamic_retry_delay:.3f}s...")
                        await asyncio.sleep(dynamic_retry_delay)
                    else:
                        df.at[index, output_column_name] = f"Error: {error_message}"
                        logging.error(f"Final failure for row {index}: {error_message}")

    # Process in Batches
    total_rows = len(df)

    if batch_size and batch_size < total_rows:
        batch_indices = np.array_split(range(total_rows), len(df) // batch_size + 1)
        batches = [df.iloc[idx] for idx in batch_indices]
    else:
        batches = [df]

    with atqdm(total=len(batches), desc="Processing Batches", leave=True) as pbar:
        for batch in batches:
            tasks = [process_row(index, row) for index, row in batch.iterrows()]
            await asyncio.gather(*tasks)

            # Save cache after each batch
            if cache_file:
                async with aiofiles.open(cache_file, "w", encoding="utf-8") as f:
                    await f.write(json.dumps(cache, ensure_ascii=False, indent=4))

            pbar.update(1)  # Update progress bar

    return df

propmpt structure:
"Detect the language of the text and return ONLY the ISO country code (e.g., en, fr, de, ect.). Text:{cell text}"

### Dealing with not enought cinfidence

In [10]:
df_result = await chatgpt_async(
    input_column_name="Job Description", 
    output_column_name="Language gpt-4o-2024-11-20",
    input_text_length=None,
    output_text_length=1,
    num_rows=None,  
    df=not_enough_confidence.copy(), 
    user_prompt="Detect the language of the text and return ONLY the ISO country code (e.g., en, fr, de, ect.). Text:",
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=10,
    concurrency_limit=10,
    cache_file="./cache/language_cache_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 2/2 [00:00<00:00, 57.32it/s]


In [11]:
df_result['Manual check'] = ["drop", "cs", "drop", "ru", "drop", "fr", "en", "pl", "en", "drop", "fr", "en", "pl", "sv", "zh", "fr"]
df_result['Drop'] = [True, False, True, False, True, False, False, False, False, True, False, False, False, False, False, False]

df_result_final = df_result.copy()
df_result_final = df_result_final.loc[~df_result_final['Drop']]

In [12]:
"""
Cell generated by Data Wrangler.
"""
def clean_data2(df_result_final):
    # Drop columns: 'Drop', 'Language langdetect' and 2 other columns
    df_result_final = df_result_final.drop(columns=['Drop', 'Language langdetect', 'Language langdetect confidence', 'Language gpt-4o-2024-11-20'])
    # Rename column 'Manual check' to 'Language'
    df_result_final = df_result_final.rename(columns={'Manual check': 'Language'})
    return df_result_final

df_result_final_clean = clean_data2(df_result_final.copy())
df_result_final_clean.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language
528,Czechia,Europe,True,True,default,google.com,Android Developer (part-time: 2h/week) @ Exper...,Experis Polska,Anywhere,"Jooble, Jobs Trabajo.org",O pozici / o projektu\r\nPůvodní popisek. Andr...,True,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciAocG...,2025-01-13 12:09:27 UTC,Android developer,cs
780,Germany,Europe,True,True,default,google.com,JUNIOR IOS DEVELOPER,Check24,Germany,Layboard,Требования 1-2 years of experience with iOS de...,NaN,€4.5K a month,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJKVU5JT1IgSU9TIERFVkVMT1BFUi...,2025-01-13 12:22:03 UTC,iOS developer,ru
325,Canada,Northern America,False,False,default,google.com,"Développeur senior, Android / Senior Android D...",Cerence,"Montreal, Quebec, Canada","Indeed, Built In, Eluta.ca, Glassdoor, Adzuna,...",A Moving Experience.\r\n\r\n(English version b...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJEw6l2ZWxvcHBldXIgc2VuaW9yLC...,2025-01-13 12:09:10 UTC,Android developer,fr
2105,Sweden,Europe,True,True,default,google.com,Konsultuppdrag Ios and Android Developers - Of...,Senterprise,ستوكهولم، السويد,Emprego.pt,To one of our clients we are now looking for 1...,NaN,NaN,دوام كامل,NaN,eyJqb2JfdGl0bGUiOiJLb25zdWx0dXBwZHJhZyBJb3MgYW...,2025-01-13 12:11:47 UTC,Android developer,en
1591,Poland,Europe,True,True,default,google.com,Android Developer,ALAN Systems,"Silesian Voivodeship, Poland",JobLeads,"Cześć odkrywco kodu!\r\n\r\nCzy jesteś gotowy,...",NaN,PLN 180K–PLN 240K a year,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciIsIm...,2025-01-13 12:11:12 UTC,Android developer,pl


### Merging

In [13]:
df_language = pd.concat([sample_sections_test, df_result_final_clean], ignore_index=True)
df_language = df_language.sort_values(['Location'])
df_language.reset_index(inplace=True, drop=True)
df_language.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,Job Description,Work from home,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",We are a dynamic FinTech company headquartered...,NaN,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",Lead Mobile Developer | Hyper Growth Startup |...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJNb2JpbGUgQXBwbGljYXRpb24gRG...,2025-01-13 12:08:51 UTC,Android developer,en
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,Overview:\r\nThe Android Developer – Kotlin po...,NaN,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciDigJ...,2025-01-13 12:08:48 UTC,Android developer,en
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,Was sind deine Aufgaben?\r\n• Du gestaltest at...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:20:43 UTC,iOS developer,de
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",Do you want to push the frontier of digital se...,NaN,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIFVwMyAoZi...,2025-01-13 12:20:39 UTC,iOS developer,en


## Translate Job Descriptions into English 

Original:
This is a [SRC] to [TGT]
translation, please provide
the [TGT] translation for these
sentences:


My version:
This is a [SRC] to [TGT]
translation, please provide
the [TGT] translation for this
job description:

In [14]:
df_language = df_language.copy()

df_language['Language'].unique()

array(['en', 'de', 'it', 'fr', 'nl', 'es', 'hr', 'ru', 'bg', 'cs', 'pl',
       'uk', 'da', 'fi', 'hu', 'lv', 'zh', 'pt', 'ro', 'sk', 'sl', 'sv'],
      dtype=object)

In [15]:
language_mapping = {
    'en': 'English',
    'de': 'German',
    'it': 'Italian',
    'fr': 'French',
    'nl': 'Dutch',
    'es': 'Spanish',
    'hr': 'Croatian',
    'ru': 'Russian',
    'bg': 'Bulgarian',
    'cs': 'Czech',
    'pl': 'Polish',
    'uk': 'Ukrainian',
    'da': 'Danish',
    'fi': 'Finnish',
    'hu': 'Hungarian',
    'lv': 'Latvian',
    'zh': 'Chinese',
    'pt': 'Portuguese',
    'ro': 'Romanian',
    'sk': 'Slovak',
    'sl': 'Slovenian',
    'sv': 'Swedish'
}

df_language['Language Name'] = df_language['Language'].map(language_mapping)


In [16]:
async def translate_non_english_descriptions(df, language_col, job_desc_col, translated_col, client, gpt_model, language_mapping, batch_size=10, concurrency_limit=10, cache_file="./cache/job_description_english_cache.json"):
    """
    Translates non-English job descriptions into English using chatgpt_async().
    Copies English job descriptions directly without translation.

    Parameters:
    - df: DataFrame containing job descriptions and their languages.
    - language_col: Column name for ISO language codes.
    - job_desc_col: Column name for job descriptions.
    - translated_col: Column name where translated descriptions will be stored.
    - client: OpenAI async client instance.
    - gpt_model: GPT model name.
    - batch_size: Number of rows to process per batch.
    - concurrency_limit: Max concurrent API requests.
    - cache_file: Path to store cache.

    Returns:
    - df: Updated DataFrame with translated job descriptions.
    """
    if language_mapping is None:
        raise ValueError("language_mapping(key-value pairs) must be provided.")
    # Language mapping dictionary

    # Ensure the translated column exists
    if translated_col not in df.columns:
        df[translated_col] = ""

    # Step 1: Copy English job descriptions (skip API calls for them)
    df.loc[df[language_col] == "en", translated_col] = df.loc[df[language_col] == "en", job_desc_col]

    # Step 2: Filter only non-English rows that still need translation
    mask_non_english = (df[language_col] != "en") & (df[translated_col] == "")
    
    if not mask_non_english.any():
        return df  # Nothing to translate

    # Step 3: Generate dynamic prompts directly using .loc[] (avoiding SettingWithCopyWarning)
    df.loc[mask_non_english, "user_prompt"] = df.loc[mask_non_english, language_col].map(
        lambda lang: f"This is a {language_mapping.get(lang, 'Unknown')} to English translation, please provide the English translation for this job description:"
    )

    # Step 4: Run chatgpt_async() for translation
    df_translated = await chatgpt_async(
        input_column_name=job_desc_col,
        output_column_name=translated_col,
        df=df.loc[mask_non_english],  # No .copy(), keeping reference to df
        user_prompt="user_prompt",  # Dynamic per row
        gpt_model=gpt_model,
        client=client,
        batch_size=batch_size,
        concurrency_limit=concurrency_limit,
        cache_file=cache_file
    )

    # Step 5: Merge results back into original DataFrame
    df.update(df_translated)

    # Step 6: Remove the "user_prompt" column
    df.drop(columns=["user_prompt"], inplace=True, errors="ignore")

    return df

In [17]:
df_translated = await translate_non_english_descriptions(
    df=df_language.copy(),
    language_col="Language",
    job_desc_col="Job Description",
    translated_col="Job Description English",
    client=client_async,
    gpt_model="gpt-4o-2024-11-20",
    batch_size=10,
    concurrency_limit=10,
    language_mapping=language_mapping,
    cache_file="./cache/job_description_english_cache_V2_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 42/42 [00:01<00:00, 34.16it/s]


### Drop too short Job Descriptions

In [18]:
df_translated = df_translated[df_translated['Job Description English'].str.split().str.len() >= 14].reset_index(drop=True)

## Extract the information from the "Job Description in English" into logical sections

### Ground Truth testing
#### Dataframe into Text

In [19]:
sample = df_translated.sample(n=30, random_state=42)

def column_to_txt(column, output, df):
    separator = "\n\n\n" + "-" * 100 + "\n\n\n"
    with open(output, "w", encoding="utf-8") as f:
        f.write(separator.join(df[column].tolist()))

#column_to_txt(column='Job Description English', output="./ground truth/job_descriptions_sample.txt", df=sample)

####  Load Ground Truth into a DataFrame

In [20]:
def txt_to_column(txt_path, df):
    df = df.copy()
    with open(txt_path, "r", encoding="utf-8") as f:
        content = f.read().strip()  # Read and remove extra spaces
    ground_truth_labels = content.split("-" * 100)
    ground_truth_labels = [label.strip("\n") for label in ground_truth_labels]
    
    if len(ground_truth_labels) > len(df):
        ground_truth_labels = ground_truth_labels[:len(df)]
        df["Ground Truth"] = ground_truth_labels
    else:
        df["Ground Truth"] = ground_truth_labels
    return df

test = txt_to_column(txt_path="./ground truth/job_descriptions_ground_truth.txt", df=sample)
test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Language Name,Job Description English,Ground Truth
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciIsIm...,2025-01-13 12:11:12 UTC,Android developer,pl,Polish,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIChSZW1vdG...,2025-01-13 12:21:02 UTC,iOS developer,en,English,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJJT1MgZGV2ZWxvcGVyIiwiY29tcG...,2025-01-13 12:22:59 UTC,iOS developer,nl,Dutch,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\..."
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciAvIE...,2025-01-13 12:11:28 UTC,Android developer,en,English,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJFbWJlZGRlZCBTb2Z0d2FyZSBEZX...,2025-01-13 12:12:03 UTC,Android developer,en,English,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...


#### Extract sections from Ground Truth

In [21]:
# Function to extract each section
def extract_section(text, section_name):
    pattern = rf"{section_name}:\s*(.*?)(?=\n\d+\.|\Z)"
    match = re.search(pattern, text, re.DOTALL)
    return match.group(1).strip() if match else None

#Apply to all rowsr
test["Platform_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"1\. Platform"))
test["Salary_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"2\. Salary"))
test["Requirements_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"3\. Requirements"))
test["Nice to have_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"4\. Nice to have"))
test["Responsibilities_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"5\. Responsibilities"))
test["Benefits_GT"] = test["Ground Truth"].apply(lambda x: extract_section(x, r"6\. Benefits"))


test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Ground Truth,Platform_GT,Salary_GT,Requirements_GT,Nice to have_GT,Responsibilities_GT,Benefits_GT
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,pl,Polish,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,en,English,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....,iOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,nl,Dutch,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift\n - B...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,en,English,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Deep expertise in Google CTS tests\n - Stro...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,en,English,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...,Android,£40-50K plus benefits,- Degree in relevant subject\n - Embedded rea...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...


#### Estract Sections From GPT's version

In [22]:
system_prompt = """
You are an AI assistant. Your role is to extract specific information from job descriptions and format them in a strict structure.
"""

user_prompt = """
I will provide a job description. Please extract and present the information in **this exact order**:

1. Platform: (Android/iOS/Cross-platform)
2. Salary: (If stated; otherwise "Not mentioned")
3. Requirements: (verbatim from the job description or "Not mentioned")
4. Nice to have: (verbatim or "Not mentioned")
5. Responsibilities: (verbatim or "Not mentioned")
6. Benefits: (verbatim or "Not mentioned")

**Guidelines**:
- **DO NOT reword, paraphrase, or summarize** any part of the job description. Copy the sentences exactly as they appear.
- Combine all mandatory or required skill sections (e.g., "Requirements," "Skills," "Key Technologies," "About You") under **Requirements**.
- If the job description specifically says something is "a plus," "beneficial," or otherwise indicates it’s optional, place it under "Nice to have" even if it appears under a "Requirements" heading in the job description.
- If there is an "About you" or "About Role" section (or similar) that describs duties or tasks, include those under "Responsibilities".
- If the information is not in the job description, write "Not mentioned" for that section.
- For multiple platforms (e.g., Android, iOS), list them all in **Platform** and use headings under Requirements (and other sections, if needed) like "General Requirements:", "For Android Developers:", "For iOS Developers:".
- Present your answer **only** in the format above.

---
Here is the job description:
"""

df_extracted_test = await chatgpt_async(
    input_column_name="Job Description English", 
    output_column_name="Job Description Extracted",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=test.copy(), 
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=5,
    concurrency_limit=10,
    cache_file="./cache/job_description_extracted_cache_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 7/7 [00:01<00:00,  6.65it/s]


In [23]:
df_extracted_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language Name,Job Description English,Ground Truth,Platform_GT,Salary_GT,Requirements_GT,Nice to have_GT,Responsibilities_GT,Benefits_GT,Job Description Extracted
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,Polish,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...,1. Platform: Android \n2. Salary: Not mention...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,English,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS\n2. Salary: Not mentioned\n3....,iOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned,1. Platform: iOS/macOS \n2. Salary: Not menti...
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,Dutch,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS\n2. Salary: €3,200 to €5,000\...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift\n - B...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...,"1. Platform: iOS \n2. Salary: €3,200 to €5,00..."
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,English,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android\n2. Salary: Not mentioned...,Android,Not mentioned,- Deep expertise in Google CTS tests\n - Stro...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned,1. Platform: Android \n2. Salary: Not mention...
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,English,If you have experience developing embedded rea...,1. Platform: Android\n2. Salary: £40-50K plus ...,Android,£40-50K plus benefits,- Degree in relevant subject\n - Embedded rea...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...,1. Platform: Android \n2. Salary: £40-50K plu...


In [24]:
#Apply to all rowsr
df_extracted_test["Platform_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"1\. Platform"))
df_extracted_test["Salary_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"2\. Salary"))
df_extracted_test["Requirements_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"3\. Requirements"))
df_extracted_test["Nice to have_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"4\. Nice to have"))
df_extracted_test["Responsibilities_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"5\. Responsibilities"))
df_extracted_test["Benefits_GPT"] = df_extracted_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"6\. Benefits"))


df_extracted_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Nice to have_GT,Responsibilities_GT,Benefits_GT,Job Description Extracted,Platform_GPT,Salary_GPT,Requirements_GPT,Nice to have_GPT,Responsibilities_GPT,Benefits_GPT
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,- Diving deep into lower-level libraries like ...,"(list out or say ""Not mentioned"")\n - Your wo...",Not mentioned,1. Platform: iOS/macOS \n2. Salary: Not menti...,iOS/macOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,- Dive deep into anti-censorship technologies ...,Not mentioned
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,- Experience in an environment where cybersecu...,- Primarily focus on backend development for t...,- 28 vacation days + the option to purchase 10...,"1. Platform: iOS \n2. Salary: €3,200 to €5,00...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift \n ...,- Experience in an environment where cybersecu...,- You’ll primarily focus on backend developmen...,- 28 vacation days + the option to purchase 10...
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,- Docker\n - AUTOSAR (AUTomotive Open System ...,- Work inside an intercultural team of many la...,Not mentioned,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Deep expertise in Google CTS tests \n - S...,- Docker \n - AUTOSAR (AUTomotive Open Syst...,- Work inside an intercultural team of many la...,Not mentioned
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,- Experience of Linux kernel and system progra...,Not mentioned,- An early finish on Fridays\n - Bonus Privat...,1. Platform: Android \n2. Salary: £40-50K plu...,Android,£40-50K plus benefits,- Degree in relevant subject \n - Embedded ...,- Experience of Linux kernel and system progra...,- Develop and support the software running on ...,- An early finish on Fridays \n - Bonus \n...


#### Calculate Fuzzy Scores

1. Token Sort Ratio (Handling Word Order Differences)<br>
If two sentences contain the same words but in different order, Levenshtein distance alone might give a low similarity score. Instead, we:
- Split both sentences into words.
- Sort them alphabetically.
- Recalculate Levenshtein Distance.

2. Token Set Ratio (Handling Partial Overlaps)<br>
If one string is a subset of another, Token Set Ratio helps:
- Convert both sentences into sets of unique words.
- Compare only the common words.

In [25]:
def compare_fuzzy_sections(df):
    sections = ['Platform', 'Salary', 'Requirements', 'Nice to have', 'Responsibilities', 'Benefits']
    
    results = []

    for section in sections:
        col_gt = f"{section}_GT"
        col_gpt = f"{section}_GPT"
        
        df[f"{section} Token Set Ratio"] = df.apply(
            lambda row: fuzz.token_set_ratio(str(row[col_gt]), str(row[col_gpt])), axis=1
        ).round(2)
        
        avg_score = df[f"{section} Token Set Ratio"].mean().round(2)
        results.append((section, avg_score))

    print("Average Token Set Ratio per section:")
    for section, score in results:
        print(f"{section}: {score}")

    values = [value for _, value in results]
    print(f"\nOverall Average Token Set Ratio: {np.mean(values):.2f}")

compare_fuzzy_sections(df_extracted_test)


Average Token Set Ratio per section:
Platform: 98.33
Salary: 97.35
Requirements: 98.61
Nice to have: 93.38
Responsibilities: 90.88
Benefits: 91.85

Overall Average Token Set Ratio: 95.07


### Full Extraction 

In [26]:
df_extracted = await chatgpt_async(
    input_column_name="Job Description English", 
    output_column_name="Job Description Extracted",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=df_translated.copy(), 
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=30,
    concurrency_limit=35,
    cache_file="./cache/job_description_extracted_cache_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 95/95 [00:13<00:00,  7.05it/s]


In [27]:
df_extracted.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Salary,Schedule type,Qualifications,Job ID,Search Date,Search Query,Language,Language Name,Job Description English,Job Description Extracted
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,NaN,Full-time,NaN,eyJqb2JfdGl0bGUiOiJTYW5kYm94IElubm92YXRpb24gU2...,2025-01-13 12:33:21 UTC,iOS developer,en,English,We are a dynamic FinTech company headquartered...,"1. Platform: Android, iOS, Cross-platform \n2..."
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJNb2JpbGUgQXBwbGljYXRpb24gRG...,2025-01-13 12:08:51 UTC,Android developer,en,English,Lead Mobile Developer | Hyper Growth Startup |...,1. Platform: Cross-platform \n2. Salary: $160...
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,NaN,Contractor,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIERldmVsb3BlciDigJ...,2025-01-13 12:08:48 UTC,Android developer,en,English,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJBbmRyb2lkIFx1MDAyNiBpT1MgRG...,2025-01-13 12:20:43 UTC,iOS developer,de,German,**What are your tasks?** \n- You design attra...,1. Platform: Android/iOS \n2. Salary: Not men...
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",...,NaN,Full-time,No degree mentioned,eyJqb2JfdGl0bGUiOiJpT1MgRGV2ZWxvcGVyIFVwMyAoZi...,2025-01-13 12:20:39 UTC,iOS developer,en,English,Do you want to push the frontier of digital se...,1. Platform: iOS \n2. Salary: The gross annua...


## Extracting sections into columns

### Testing

In [28]:
df_sections = df_extracted.copy()

sample_sections_test = df_sections.sample(n=50, random_state=42)

In [29]:
#Apply to all rowsr
sample_sections_test["Platform"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"1\. Platform"))
sample_sections_test["Salary_E"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"2\. Salary"))
sample_sections_test["Requirements"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"3\. Requirements"))
sample_sections_test["Nice to have"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"4\. Nice to have"))
sample_sections_test["Responsibilities"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"5\. Responsibilities"))
sample_sections_test["Benefits"] = sample_sections_test["Job Description Extracted"].apply(lambda x: extract_section(x, r"6\. Benefits"))


sample_sections_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
1575,Poland,Europe,True,True,default,google.com,Android Developer,Connectis_,"Kórnik, Poland",HitPraca.pl,...,pl,Polish,**Android Developer** \n**Location:** Poznań ...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Minimum 3 years of commercial experience as ...,- Familiarity with technologies and tools such...,- Designing and implementing new screens in th...,- Opportunity to participate in integration ev...
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,en,English,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS/macOS \n2. Salary: Not menti...,iOS/macOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,- Dive deep into anti-censorship technologies ...,Not mentioned
1491,Netherlands,Europe,True,True,default,google.com,IOS developer,StarApple,"Eindhoven, Netherlands",Werkzoeken.nl,...,nl,Dutch,**Organization & Department**\n\nAre you inter...,"1. Platform: iOS \n2. Salary: €3,200 to €5,00...",iOS,"€3,200 to €5,000",- 3 years of work experience with Swift \n ...,- Experience in an environment where cybersecu...,- You’ll primarily focus on backend developmen...,- 28 vacation days + the option to purchase 10...
1815,Romania,Europe,True,True,default,google.com,Android Developer / CTS Expert,Luxoft,Romania,"Indeed, Jooble, LinkedIn",...,en,English,"Remote Romania, Romania\r\n\r\nAndroid\r\n\r\n...",1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,- Deep expertise in Google CTS tests \n - S...,- Docker \n - AUTOSAR (AUTomotive Open Syst...,- Work inside an intercultural team of many la...,Not mentioned
2174,United Kingdom,Europe,False,False,default,google.com,Embedded Software Developer (Android),Integrity,"Chesterton, United Kingdom",SitePoint,...,en,English,If you have experience developing embedded rea...,1. Platform: Android \n2. Salary: £40-50K plu...,Android,£40-50K plus benefits,- Degree in relevant subject \n - Embedded ...,- Experience of Linux kernel and system progra...,- Develop and support the software running on ...,- An early finish on Fridays \n - Bonus \n...


### Deployment

In [30]:
df_sections["Platform"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"1\. Platform"))
df_sections["Salary_E"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"2\. Salary"))
df_sections["Requirements"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"3\. Requirements"))
df_sections["Nice to have"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"4\. Nice to have"))
df_sections["Responsibilities"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"5\. Responsibilities"))
df_sections["Benefits"] = df_sections["Job Description Extracted"].apply(lambda x: extract_section(x, r"6\. Benefits"))


df_sections.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
0,Austria,Europe,True,True,local,google.at,Sandbox Innovation Sdn. Bhd. | Senior Mobile A...,Tideri Jobbörse,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,en,English,We are a dynamic FinTech company headquartered...,"1. Platform: Android, iOS, Cross-platform \n2...","Android, iOS, Cross-platform",Not mentioned,- Educational Background: Bachelor's degree in...,- Flutter experience is a plus. \n - Progra...,"- Develop, test, and deploy high quality mobil...",- Young & dynamic workplace & culture (with of...
1,Austria,Europe,True,True,default,google.com,Mobile Application Developer,Pearson Carter,Austria,"Trabajo.org - Stellenangebote, Arbeit",...,en,English,Lead Mobile Developer | Hyper Growth Startup |...,1. Platform: Cross-platform \n2. Salary: $160...,Cross-platform,"$160,000 + Super",Not mentioned,Not mentioned,Not mentioned,Not mentioned
2,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,en,English,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Passionate about mobile platforms and transl...,Not mentioned,• Ensure that the app meets our quality standa...,Not mentioned
3,Austria,Europe,True,True,default,google.com,Android & iOS Developer,ventopay gmbh,Austria,StudySmarter - Talents,...,de,German,**What are your tasks?** \n- You design attra...,1. Platform: Android/iOS \n2. Salary: Not men...,Android/iOS,Not mentioned,- A completed IT education (HTL/FH/University)...,- Professional experience in developing native...,- You design attractive modules and software p...,"- We are a stable, owner-managed company with ..."
4,Austria,Europe,True,True,default,google.com,iOS Developer Up3 (f/m/d),Drei Österreich,"Vienna, Austria","MyAbility.jobs, Drei., Jobted.at",...,en,English,Do you want to push the frontier of digital se...,1. Platform: iOS \n2. Salary: The gross annua...,iOS,The gross annual salary according to the colle...,• At least 3 years of experience as iOS develo...,Not mentioned,• Develop the iOS app of our fully digital pro...,• Top mobile phone of your choice incl. employ...


## Take only iOS and Android vacancies

In [31]:
platform_counts = df_sections["Platform"].value_counts().reset_index()
platform_counts.columns = ['Platform', 'Count']
print(platform_counts)

                        Platform  Count
0                            iOS   1094
1                        Android   1080
2                   Android, iOS    156
3   Android, iOS, Cross-platform    109
4                  Not mentioned     77
..                           ...    ...
67   iOS, Android, Windows Phone      1
68        Android/Cross-platform      1
69       Android, iOS, Web-based      1
70               Android, Huawei      1
71           Android, macOS, iOS      1

[72 rows x 2 columns]


In [32]:
df_sections[df_sections['Platform'] == 'Not mentioned']

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
11,Austria,Europe,True,True,default,google.com,Senior Django+Angular Developer (Full-Stack or...,iRonin - Web DevOps Mobile Developers: Ruby on...,Austria,StudySmarter - Talents,...,en,English,Senior Django+Angular Developer (Full-Stack or...,1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,- Experience: Minimum 5 years in software deve...,Not mentioned,- Enhance our legacy product catalog applicati...,- Flexible working hours. The most important t...
142,Bulgaria,Europe,True,True,local,google.bg,Mid/Senior Android Developer,Easy Consult Ltd,Anywhere,Jobgether,...,en,English,"This a Full Remote job, the offer is available...",1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,- 4+ years of experience in Kotlin development...,Not mentioned,- ﻿﻿Actively participate in development relate...,- Simple recruitment process as well as quick ...
319,Canada,Northern America,False,False,default,google.com,IOS Developer,Bounteous,"Calgary, AB, Canada","Startup Jobs, Jobs Trabajo.org",...,en,English,Opening from Canada\r\nBounteous x Accolite ma...,1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned,Not mentioned
383,Canada,Northern America,False,False,default,google.com,iPhone / iPad & Android Developer,eMacity Leads,"Ontario, Canada","Indeed, Glassdoor, SimplyHired",...,en,English,Level of experience: Middle/High\r\n\r\nPositi...,1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,To analyse and implement highly efficient webs...,"Joomla, WordPress as a plus.",Not mentioned,Not mentioned
395,Canada,Northern America,False,False,default,google.com,Experienced iOS Developer Wanted for Large-Sca...,"LGS, une Société IBM an IBM Company","Montreal, Quebec, Canada",Jobs Trabajo.org,...,en,English,"LGS, an IBM Company, is committed to providing...","1. Platform: Not mentioned \n2. Salary: $120,...",Not mentioned,"$120,000 - $180,000 per year, depending on exp...",Not mentioned,Not mentioned,- Participate in the development of client pro...,Not mentioned
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,United States,Northern America,False,False,default,google.com,SWE - Tools & Automation Engineer - Developer ...,Apple,"San Diego, CA","Careers At Apple, AARP Job Board, ZipRecruiter...",...,en,English,"Summary\r\nPosted: Oct 16, 2024\r\nWeekly Hour...","1. Platform: Not mentioned \n2. Salary: $129,...",Not mentioned,"$129,600 - $236,300",• Excellent understanding of the software deve...,"• Knowledge of iOS, macOS, and Xcode. \n • ...",- Collaborate closely with software developers...,"• At Apple, base pay is one part of our total ..."
2724,United States,Northern America,False,False,default,google.com,Android Developer/ Ios Developer,Keylent Inc,"Detroit, MI","ZipRecruiter, Smart Recruiters Jobs, Ladders",...,en,English,Company Description\r\n\r\nWe established Keyl...,1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,• Strong Object Oriented development backgroun...,Not mentioned,Not mentioned,Not mentioned
2753,United States,Northern America,False,False,default,google.com,Sr. iOS Developer,FlexIT Inc,"Beaverton, OR","Indeed, ZipRecruiter, SimplyHired, BeBee, Lear...",...,en,English,RESPONSIBILITIES\r\n• Work with talented engin...,1. Platform: Not mentioned \n2. Salary: Not m...,Not mentioned,Not mentioned,Not mentioned,Not mentioned,• Work with talented engineers for the technic...,Not mentioned
2782,United States,Northern America,False,False,default,google.com,"Senior iOS Developer, Payments",Score Media and Gaming Inc.,"Philadelphia, PA","Startup Jobs, Lad

In [33]:
df_sections[(df_sections['Platform'] == 'iOS/macOS') | (df_sections['Platform'] == 'iOS, tvOS')]

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
324,Canada,Northern America,False,False,default,google.com,iOS Developer (Remote),McAfee,"Waterloo, ON, Canada",Blind,...,en,English,Role OverviewMcAfee is searching for an interm...,1. Platform: iOS/macOS \n2. Salary: Not menti...,iOS/macOS,Not mentioned,- You can develop iOS applications at an inter...,- Diving deep into lower-level libraries like ...,- Dive deep into anti-censorship technologies ...,Not mentioned
385,Canada,Northern America,False,False,default,google.com,"Software Developer in Test, Creativity Apps",Apple,"Vancouver, BC, Canada",Careers At Apple,...,en,English,"Summary\r\nPosted: Sep 4, 2024\r\n\r\nRole Num...","1. Platform: iOS/macOS \n2. Salary: $113,400 ...",iOS/macOS,"$113,400 and $215,300",• 5+ years experience in QA/QE \n • Minimum...,• 2+ years of Full-stack Software Developer in...,"In this role, you will be responsible for plan...","• At Apple, base pay is one part of our total ..."
534,Czechia,Europe,True,True,default,google.com,iOS Developer,Sledovanitv.cz,"Brno, Czechia",Indeed.cz,...,cs,Czech,We are SledovaniTV.cz - the most technological...,"1. Platform: iOS, tvOS \n2. Salary: Not menti...","iOS, tvOS",Not mentioned,- Advanced knowledge of iOS development (exper...,- Your own project to showcase,- Have the opportunity to dive into our iOS an...,"- The opportunity to work in a young, inspirin..."
1273,Mexico,Northern America,False,False,default,google.com,Senior Ios Developer,Bhuvi It Solutions,"Guadalajara, Jalisco, Mexico","BeBee, Trabajo.org - Vacantes De Empleo, Traba...",...,en,English,Job Title: iOS Developer\r\n\r\nWe are seeking...,"1. Platform: iOS, tvOS \n2. Salary: $120,000 ...","iOS, tvOS","$120,000 - $180,000 per year.",• 4+ years of professional software developmen...,Not mentioned,"• Produce a reliable, performant, configurable...",• TN Visa Sponsorship. \n • USDPay. \n •...


### Rename rows with 'iOS/macOS' or 'iOS, tvOS' to iOS

In [34]:
# Replace the first two rows with 'iOS, tvOS'
df_sections.loc[df_sections[df_sections['Platform'] == 'iOS, tvOS'].iloc[:2].index, 'Platform'] = 'iOS'

# Replace the first two rows with 'iOS/macOS'
df_sections.loc[df_sections[df_sections['Platform'] == 'iOS/macOS'].iloc[:2].index, 'Platform'] = 'iOS'


### Extraction only iOS and Android Vacancies

In [35]:
df_final = df_sections.copy()

df_final = df_final[df_final['Platform'].isin(['iOS', 'Android'])]
df_final = df_final.sort_values(by='Location').reset_index(drop=True)

df_final.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits
0,Austria,Europe,True,True,default,google.com,"Android Developer – Kotlin (Austria based, Hyb...",Bitcoin Devs Company,"Vienna, Austria",Jobs3,...,en,English,Overview:\r\nThe Android Developer – Kotlin po...,1. Platform: Android \n2. Salary: Not mention...,Android,Not mentioned,• Passionate about mobile platforms and transl...,Not mentioned,• Ensure that the app meets our quality standa...,Not mentioned
1,Austria,Europe,True,True,default,google.com,ios entwickler 80–100% w/m/d,CHANCENLAND VORARLBERG,"Dornbirn, Austria","IT-Career.at, STEMJOBS.AT, IT-JOBS.AT",...,de,German,**iOS Developer 80–100% w/m/d**\n\n**Job Descr...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"- **Motivation over experience:** Curiosity, i...",Not mentioned,- You will work with us on exciting projects f...,"- A compact, battle-tested team and flat hiera..."
2,Austria,Europe,True,True,default,google.com,Middle iOS developer,Processica,"Vienna, Austria",JOBITT,...,en,English,Looking for a iOS Developer. Playing well in a...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,Playing well in a team and has strong analytic...,Not mentioned,Not mentioned,Not mentioned
3,Austria,Europe,True,True,default,google.com,iOS Developer,Raiffeisen Gruppe,"Linz, Austria","Jooble, Trabajo.org - Stellenangebote, Arbeit",...,de,German,**Your Role in the Team** \n- You contribute ...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- You are familiar with Continuous Integration...,Not mentioned,- You contribute to the further development of...,- Home Office \n - Flexible working hours ...
4,Austria,Europe,True,True,default,google.com,iOS Software Engineer,Cybermoth,"Vienna, Austria","Expertini, Talent.com",...,en,English,We are searching for iOS Software Engineers wi...,1. Platform: iOS \n2. Salary: Depending on qu...,iOS,Depending on qualifications and professional e...,• Worked on at least one native Swift applicat...,Not mentioned,• In this position you will be part of one or ...,• You will be part of a company with an inspir...


## Extract technologies and tools

### Testing

In [62]:
df_test = df_final.copy()

#df_extract_sample = df_test.sample(n=30, random_state=42)
df_extract_sample = df_test[df_test["Platform"] == "iOS"].sample(n=30, random_state=42)

In [63]:
df_extract_sample["Full Requirements"] = (
    "3. Requirements:\n" + df_extract_sample["Requirements"].astype(str) + "\n\n" +
    "4. Nice to have:\n" + df_extract_sample["Nice to have"].astype(str) + "\n\n" +
    "5. Responsibilities:\n" + df_extract_sample["Responsibilities"].astype(str)
)
df_extract_sample

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Language Name,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements
69,Belgium,Europe,True,True,default,google.com,Senior iOS Developer (Remote),Neybox Digital,Anywhere,"Wellfound, Jooble, Pangian",...,English,🚀 What's the position?\r\n\r\nThis is a Full-t...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,• 4+ years of iOS development experience in Sw...,Not mentioned,• Building new features \n • Fixing bugs \...,Not mentioned,3. Requirements:\n• 4+ years of iOS developmen...
1129,Poland,Europe,True,True,default,google.com,iOS Developer,makeitright,"Gdańsk, Poland","Indeed, Jobrapido.com",...,Polish,**English Translation:**\n\nMakeitright is a t...,"1. Platform: iOS \n2. Salary: 1,040 – 1,280 P...",iOS,"1,040 – 1,280 PLN/MD",- You have at least 3 years of commercial expe...,Not mentioned,- Developing existing applications for the iOS...,- The possibility of remote work or a hybrid m...,3. Requirements:\n- You have at least 3 years ...
97,Bulgaria,Europe,True,True,default,google.com,Senior iOS Developer - Spanish/English - Full ...,Jobs4java.eu,Anywhere,Jobgether,...,English,"This a Full Remote job, the offer is available...",1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"• College / university degree, Bacc. Ing. Comp...",Not mentioned,• Design and develop software solutions to cli...,• Attractive salary package (employees); \n ...,3. Requirements:\n• College / university degre...
1233,Portugal,Europe,True,True,default,google.com,IOS Developer,AdQuam Human Capital,"Braga, Portugal","Jooble, Empregos Trabajo.org, LinkedIn, SAPO E...",...,Portuguese,**IOS Developer**\n\n**About AdQuam Human Capi...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Minimum of 3 years of experience with RxSwif...,- Experience with MVVM is valued.,Not mentioned,Not mentioned,3. Requirements:\n- Minimum of 3 years of expe...
1630,United Kingdom,Europe,False,False,default,google.com,Senior iOS Engineer,Insight Global,"Westbury, United Kingdom",Insight Global,...,English,As a Senior Software Engineer at a large media...,1. Platform: iOS \n2. Salary: $65 to $75 per ...,iOS,$65 to $75 per hour. Exact compensation may va...,- Collaborate with cross-functional teams—Prod...,Not mentioned,- Collaborate with cross-functional teams—Prod...,Not mentioned,3. Requirements:\n- Collaborate with cross-fun...
154,Canada,Northern America,False,False,default,google.com,iOS Developer,"SyncDog, Inc.","Fredericton, NB, Canada","SyncDog, Jooble",...,English,SyncDog is looking for an experienced iOS deve...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"• BS/MS degree in Computer Science, Engineerin...",Not mentioned,• Design and build advanced applications for t...,Not mentioned,3. Requirements:\n• BS/MS degree in Computer S...
1132,Poland,Europe,True,True,default,google.com,Junior iOS Developer,Ailleron,"Rzeszów, Poland",Jobrapido.com,...,Polish,"**Required Skills:** iOS, Swift, Xcode, REST A...",1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Good knowledge of iOS SDK. \n - Proficien...,- Knowledge of mobile application security. \...,- Participating in the design of applications ...,- Subsidized sports activities. \n - Privat...,3. Requirements:\n- Good knowledge of iOS SDK....
77,Belgium,Europe,True,True,default,google.com,iOS Developer,November Five,"Antwerp, Belgium","Indeed, Jobs, Findjob24h.com",...,English,ABOUT US\r\n\r\nHey there! Before we dive into...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Have experience building iOS applications wi...,- Have experience with SwiftUI or scrum - that...,- Build powerful and performant apps by turnin...,- Aim for impact: we work towards meaningful i...,3. Requirements:\n- Have experienc

### separate iOS and Android vacancies

In [ ]:
system_prompt_1 = """
You are a senior extraction assistant.

GOAL
Return ONLY a valid JSON object that maps each technology term in the input
to exactly ONE of the keys in ALLOWED_KEYS.

STRICT RULES
• Output = JSON only, no Markdown or comments.
• Keys must come from ALLOWED_KEYS.  Omit keys with empty arrays.
• Each value = array of unique strings, KEEP original spelling/case.
• Ignore soft skills and vague nouns (e.g. communication, documentation, performance, detail, English, collaboration, problem‑solving, university).
• Do NOT hallucinate: include a word ONLY if it appears in the input text.
• If a term could fit multiple keys, choose the FIRST matching key in the ordered list below (=> deterministic).
• NEVER classify Scrum, Kanban, SAFe, Agile, XP or TDD/BDD as CI/CD or any other category.

ALLOWED_KEYS  (ordered, with examples)
languages_and_runtimes                   = Swift, Kotlin, Java, Dart, Objective‑C, Coroutines, RxSwift, Combine, GCD
ui_and_cross_platform_frameworks         = SwiftUI, UIKit, Jetpack Compose, Flutter,React Native, Xamarin, Ionic, WatchKit
architectural_patterns                   = MVVM, VIPER, Clean Architecture, MVI
dependency_injection_frameworks          = Dagger, Hilt, Koin, Swinject
build_and_dependency_management          = Gradle, CocoaPods, SPM, Bazel, Buck, Xcode
data_and_caching                         = Core Data, Room, Realm, SharedPreferences
networking_and_api                       = Retrofit, OkHttp, URLSession, Alamofire,GraphQL, REST API
backend_or_baas_integration              = Firebase, AWS Amplify, Azure Mobile, Parse
device_and_platform_services             = ARKit, HealthKit, CoreML, Core Animation,Push Notifications, BLE, NFC
third_party_sdks                         = Stripe SDK, Facebook SDK, AdMob, Google Maps
security_and_cryptography                = Keychain, Keystore, TLS pinning, OAuth2
testing_frameworks                       = XCTest, JUnit, Espresso, Mockito
debugging_or_profiling                   = Instruments, Android Profiler, LeakCanary
version_control                          = Git, SVN, Mercurial
ci_cd_and_release_automation             = GitHub Actions, Jenkins, Bitrise, fastlane
monitoring_analytics_and_crash_reporting = Crashlytics, Sentry, New Relic, Datadog
development_methodologies                = Scrum, Kanban, Agile, SAFe, TDD, BDD
testing_process_and_qa                   = test coverage, regression testing
code_quality_and_static_analysis         = SonarQube, SwiftLint, ktlint, Detekt
documentation_and_knowledge_sharing      = Swagger, OpenAPI, Javadoc, Confluence
collaboration_pm_and_design_handoff      = Jira, Trello, Asana, Figma, Zeplin
distribution_and_store_operations        = TestFlight, App Store Connect,Google Play Console
compliance_and_certifications            = SOC 2, ISO 27001, GDPR
"""


user_prompt_1 = """
Extract every technology, tool, framework, library, service or formal
methodology mentioned in the text below and output the JSON as specified.

---

INPUT TEXT:
"""


df_extracted_requirements_test = await chatgpt_async(
    input_column_name="Full Requirements", 
    output_column_name="Extracted Technologies",
    input_text_length=None,
    output_text_length=None,
    num_rows=None,  
    df=df_extract_sample.copy(), 
    system_prompt=system_prompt_1,
    user_prompt=user_prompt_1,
    gpt_model="gpt-4o-2024-11-20",
    client=client_async,
    batch_size=40,
    concurrency_limit=40,
    cache_file= None #"./cache/job_description_extracted_cache_gpt-4o-2024-11-20.json"
)

Processing Batches: 100%|██████████| 1/1 [00:04<00:00,  4.07s/it]


In [65]:
df_extracted_requirements_test.head()

,Location,Region,EU Member,Schengen Agreement,Google Domain Type,Google Domain Used,Job Title,Company Name,Job Location,Apply Options,...,Job Description English,Job Description Extracted,Platform,Salary_E,Requirements,Nice to have,Responsibilities,Benefits,Full Requirements,Extracted Technologies
69,Belgium,Europe,True,True,default,google.com,Senior iOS Developer (Remote),Neybox Digital,Anywhere,"Wellfound, Jooble, Pangian",...,🚀 What's the position?\r\n\r\nThis is a Full-t...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,• 4+ years of iOS development experience in Sw...,Not mentioned,• Building new features \n • Fixing bugs \...,Not mentioned,3. Requirements:\n• 4+ years of iOS developmen...,"{\n ""languages_and_runtimes"": [""Swift"", ""GCD""..."
1129,Poland,Europe,True,True,default,google.com,iOS Developer,makeitright,"Gdańsk, Poland","Indeed, Jobrapido.com",...,**English Translation:**\n\nMakeitright is a t...,"1. Platform: iOS \n2. Salary: 1,040 – 1,280 P...",iOS,"1,040 – 1,280 PLN/MD",- You have at least 3 years of commercial expe...,Not mentioned,- Developing existing applications for the iOS...,- The possibility of remote work or a hybrid m...,3. Requirements:\n- You have at least 3 years ...,"{\n ""languages_and_runtimes"": [""Swift""],\n ""..."
97,Bulgaria,Europe,True,True,default,google.com,Senior iOS Developer - Spanish/English - Full ...,Jobs4java.eu,Anywhere,Jobgether,...,"This a Full Remote job, the offer is available...",1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,"• College / university degree, Bacc. Ing. Comp...",Not mentioned,• Design and develop software solutions to cli...,• Attractive salary package (employees); \n ...,3. Requirements:\n• College / university degre...,"{\n ""languages_and_runtimes"": [""Swift"", ""Obje..."
1233,Portugal,Europe,True,True,default,google.com,IOS Developer,AdQuam Human Capital,"Braga, Portugal","Jooble, Empregos Trabajo.org, LinkedIn, SAPO E...",...,**IOS Developer**\n\n**About AdQuam Human Capi...,1. Platform: iOS \n2. Salary: Not mentioned ...,iOS,Not mentioned,- Minimum of 3 years of experience with RxSwif...,- Experience with MVVM is valued.,Not mentioned,Not mentioned,3. Requirements:\n- Minimum of 3 years of expe...,"{\n ""languages_and_runtimes"": [""RxSwift"", ""Co..."
1630,United Kingdom,Europe,False,False,default,google.com,Senior iOS Engineer,Insight Global,"Westbury, United Kingdom",Insight Global,...,As a Senior Software Engineer at a large media...,1. Platform: iOS \n2. Salary: $65 to $75 per ...,iOS,$65 to $75 per hour. Exact compensation may va...,- Collaborate with cross-functional teams—Prod...,Not mentioned,- Collaborate with cross-functional teams—Prod...,Not mentioned,3. Requirements:\n- Collaborate with cross-fun...,"{\n ""languages_and_runtimes"": [""Swift"", ""Obje..."


In [66]:
df_extracted_requirements_test.columns

Index(['Location', 'Region', 'EU Member', 'Schengen Agreement',
       'Google Domain Type', 'Google Domain Used', 'Job Title', 'Company Name',
       'Job Location', 'Apply Options', 'Job Description', 'Work from home',
       'Salary', 'Schedule type', 'Qualifications', 'Job ID', 'Search Date',
       'Search Query', 'Language', 'Language Name', 'Job Description English',
       'Job Description Extracted', 'Platform', 'Salary_E', 'Requirements',
       'Nice to have', 'Responsibilities', 'Benefits', 'Full Requirements',
       'Extracted Technologies'],
      dtype='object')

In [67]:
output = df_extracted_requirements_test[['Full Requirements', 'Extracted Technologies']]
output

,Full Requirements,Extracted Technologies
69,3. Requirements:\n• 4+ years of iOS developmen...,"{\n ""languages_and_runtimes"": [""Swift"", ""GCD""..."
1129,3. Requirements:\n- You have at least 3 years ...,"{\n ""languages_and_runtimes"": [""Swift""],\n ""..."
97,3. Requirements:\n• College / university degre...,"{\n ""languages_and_runtimes"": [""Swift"", ""Obje..."
1233,3. Requirements:\n- Minimum of 3 years of expe...,"{\n ""languages_and_runtimes"": [""RxSwift"", ""Co..."
1630,3. Requirements:\n- Collaborate with cross-fun...,"{\n ""languages_and_runtimes"": [""Swift"", ""Obje..."
154,3. Requirements:\n• BS/MS degree in Computer S...,"{\n ""languages_and_runtimes"": [""Objective-C""]..."
1132,3. Requirements:\n- Good knowledge of iOS SDK....,"{\n ""languages_and_runtimes"": [""Swift""],\n ""..."
77,3. Requirements:\n- Have experience building i...,"{\n ""languages_and_runtimes"": [""Swift""],\n ""..."
166,3. Requirements:\n• 10+ Yrs of exp on Mobile A...,"{\n ""languages_and_runtimes"": [""iOS""]\n}"
1313,3. Requirements:\n• Experience: 3+ years of pr...,"{\n ""languages_and_runtimes"": [""Swift"", ""Obje..."


In [38]:
print(df_extract_sample["Full Requirements"].iloc[2])

3. Requirements:
• Ability to design and implement iOS applications with Xcode  
   • Proficient in Objective-C  
   • Knowledge of REST APIs  
   • Attention to details  
   • Experience working collaboratively with designers

4. Nice to have:
• Experience with Phonegap, HTML5  
   • Adept at hacking open source software  
   • Worked with git, subversion

5. Responsibilities:
Not mentioned
